In [1]:
import numpy as np
import pandas as pd
# from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder


data = pd.read_csv('test_data.csv')
data.head()


,Age,Income,CreditScore,LoanAmount,EmploymentStatus,Married,ExistingLoans,LoanApproved
0,59,81158,342,357655,SelfEmployed,No,0,0
1,49,54782,432,365387,Salaried,No,0,1
2,35,105709,694,427275,Salaried,No,0,1
3,28,98533,364,340894,SelfEmployed,Yes,0,1
4,41,128939,589,272682,Salaried,Yes,0,1


In [17]:


label_encoder_married = LabelEncoder()
data['Married'] = label_encoder_married.fit_transform(data['Married'])


In [18]:
one_hot_encoded_EmpStatus = OneHotEncoder()
EmpStatus_Encoder = one_hot_encoded_EmpStatus.fit_transform(data[['EmploymentStatus']]).toarray()

EmpStatus_Encoder_df = pd.DataFrame(EmpStatus_Encoder, columns=one_hot_encoded_EmpStatus.get_feature_names_out(['EmploymentStatus']))

# EmpStatus_Encoder_df

data = pd.concat([data.drop('EmploymentStatus', axis=1), EmpStatus_Encoder_df], axis=1)
data

,Age,Income,CreditScore,LoanAmount,Married,ExistingLoans,LoanApproved,EmploymentStatus_Salaried,EmploymentStatus_SelfEmployed,EmploymentStatus_Unemployed
0,59,81158,342,357655,0,0,0,0.0,1.0,0.0
1,49,54782,432,365387,0,0,1,1.0,0.0,0.0
2,35,105709,694,427275,0,0,1,1.0,0.0,0.0
3,28,98533,364,340894,1,0,1,0.0,1.0,0.0
4,41,128939,589,272682,1,0,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4995,36,113800,608,120779,1,2,1,1.0,0.0,0.0
4996,25,94327,744,273628,1,0,1,0.0,0.0,1.0
4997,24,127009,609,400588,0,1,1,1.0,0.0,0.0
4998,41,107911,725,154448,0,0,1,1.0,0.0,0.0


In [19]:
from sklearn.model_selection import train_test_split

x = data.drop('LoanApproved', axis=1)
y = data['LoanApproved']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

Scaler = StandardScaler()
x_train = Scaler.fit_transform(x_train)
x_test = Scaler.transform(x_test)

# x_train

In [20]:
import pickle


with open('label_encoder_married.pkl', 'wb') as file:
    pickle.dump(label_encoder_married, file)


with open('one_hot_encoded_EmpStatus.pkl', 'wb') as file:
    pickle.dump(one_hot_encoded_EmpStatus, file)

with open('Scaler.pkl', 'wb') as file:
    pickle.dump(Scaler, file)



In [21]:
x_train

array([[-0.8189351 , -0.70822378, -0.62248627, ..., -1.26321695,
         1.56564718, -0.32493568],
       [ 0.52145734,  0.47028377, -1.49958045, ...,  0.79162966,
        -0.63871351, -0.32493568],
       [ 0.34273835,  0.93624111, -1.3868112 , ...,  0.79162966,
        -0.63871351, -0.32493568],
       ...,
       [-1.62317057,  0.83353165,  1.54518935, ...,  0.79162966,
        -0.63871351, -0.32493568],
       [ 1.4150523 ,  0.77239381, -0.21526397, ...,  0.79162966,
        -0.63871351, -0.32493568],
       [ 1.32569281,  1.23341983,  0.89989863, ..., -1.26321695,
         1.56564718, -0.32493568]], shape=(4000, 9))

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])




/home/tarun/projects/generative_course_practice/venv/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


tensorboard_callback=TensorBoard(log_dir="logs/fit/",histogram_freq=1)
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [25]:
model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7588 - loss: 0.5015 - val_accuracy: 0.7460 - val_loss: 0.5167
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7620 - loss: 0.4998 - val_accuracy: 0.7420 - val_loss: 0.5176
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7630 - loss: 0.4978 - val_accuracy: 0.7430 - val_loss: 0.5201
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7635 - loss: 0.4975 - val_accuracy: 0.7390 - val_loss: 0.5217
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7632 - loss: 0.4963 - val_accuracy: 0.7470 - val_loss: 0.5188
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7640 - loss: 0.4945 - val_accuracy: 0.7440 - val_loss: 0.5219
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7635 - loss: 0.4938 - val_accuracy: 0.7430 - val_loss: 0.5217
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7663 - loss: 0.4927 - val_accuracy: 0.7470

In [26]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,261 (32.27 KB)

 Trainable params: 2,753 (10.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,508 (21.52 KB)

In [29]:
model.save('model.h5')

In [30]:
model.evaluate(x_test,y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7460 - loss: 0.5167


[0.5167080163955688, 0.7459999918937683]